<a href="https://colab.research.google.com/github/sovunia-hub/mathematical-modeling-of-applied-problems/blob/main/5_stabilized_biconjugate_gradient_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

an_sol = lambda x: 3/4 * x

In [ ]:
def findAF(N):
  a = 0
  b = 1
  h = np.abs(b - a) / N
  x = np.arange(N) * h + h / 2 + a
  A = np.zeros((N, N))
  for i in range(N):
    for j in range(N):
      if i == j:
        A[i, j] = h * x[i] * x[j] + 1
      else:
        A[i, j] = h * x[i] * x[j]
  return A, x

In [ ]:
def findSimpleU(N, eps):
  A, f = findAF(N)
  I = np.eye(N)
  u = np.zeros(N)
  B = I - A
  f_norm = np.linalg.norm(f)
  kol = 0
  while True:
    kol += 1
    u_prev = u
    u = B @ u + f
    if np.linalg.norm(u - u_prev) / f_norm < eps:
      return u, kol

In [ ]:
def findGradU(N, eps):
  A , f = findAF(N)
  u = np.zeros(N)
  f_norm = np.linalg.norm(f)
  kol = 0
  while True:
    kol += 1
    r = A @ u - f
    Ar = A.conj().T @ r
    u_prev = u
    u = u_prev - (np.linalg.norm(Ar)**2 / np.linalg.norm(A @ Ar)**2) * Ar
    if np.linalg.norm(u - u_prev) / f_norm < eps:
      return u, kol

In [ ]:
def findDoubleGradU(N, eps):
  A, f = findAF(N)
  u = np.zeros(N)
  r = A @ u - f
  u_prev = u
  Ar = A.conj().T @ r
  u = u_prev - (np.linalg.norm(Ar)**2 / np.linalg.norm(A @ Ar)**2) * Ar
  f_norm = np.linalg.norm(f)
  kol = 1
  while True:
    kol += 1
    r_prev = r
    r = A @ u - f
    Ar = A.conj().T @ r
    u_prev_prev = u_prev
    u_prev = u
    Ar_norm_square = np.linalg.norm(Ar) ** 2
    alpha_gamma = np.linalg.solve(np.array([[np.linalg.norm(r - r_prev) ** 2, Ar_norm_square],
                                            [Ar_norm_square, np.linalg.norm(A @ Ar) ** 2]]),
                                  np.array([0, Ar_norm_square]))
    u = u_prev - alpha_gamma.item(0) * (u_prev - u_prev_prev) - alpha_gamma.item(1) * Ar
    if np.linalg.norm(u - u_prev) / f_norm < eps:
      return u, kol

In [ ]:
def findBiconjGradU(N, eps):
  A, f = findAF(N)
  u = np.zeros(N)
  f_norm = np.linalg.norm(f)

  r = f - A @ u
  temp_r = r
  ro = alp = om = 1
  v = p = s = np.zeros(N)
  kol = 0

  while True:
    kol += 1

    prev_ro = ro
    ro = sum(np.conj(temp_r) * r)
    bet = ro / prev_ro * (alp / om)
    p = r + bet * (p - om * v)
    v = A @ p
    alp = ro / sum(np.conj(temp_r) * v)
    s = r - alp * v
    t = A @ s
    om = sum(t * s) / sum(t * t)
    u_prev = u
    u = u_prev + om * s + alp * p
    r = s - om * t

    if np.linalg.norm(u - u_prev) / f_norm < eps:
      return u, kol

In [ ]:
N = 1000
eps = 1e-8

a = 0
b = 1
h = np.abs(b - a) / N
x = np.arange(N) * h + h / 2 + a
solution = an_sol(x)

%time uSimple, kolSimple = findSimpleU(N, eps)
print(f'Метод простых итераций: {kolSimple}')
print(f'Ошибка: {sum(np.abs(solution - uSimple)) / sum(np.abs(solution))}\n')

%time uGrad, kolGrad = findGradU(N, eps)
print(f'Метод градиентного спуска: {kolGrad}')
print(f'Ошибка: {sum(np.abs(solution - uGrad)) / sum(np.abs(solution))}\n')

%time uDoubleGrad, kolDoubleGrad = findDoubleGradU(N, eps)
print(f'Двухшаговый метод градиентного спуска: {kolDoubleGrad}')
print(f'Ошибка: {sum(np.abs(solution - uDoubleGrad)) / sum(np.abs(solution))}\n')

%time uBiconj, kolBiconj = findBiconjGradU(N, eps)
print(f'Метод бисопряженных градиентов: {kolBiconj}')
print(f'Ошибка: {sum(np.abs(solution - uBiconj)) / sum(np.abs(solution))}\n')

CPU times: user 1 s, sys: 31.8 ms, total: 1.03 s
Wall time: 2.16 s
Метод простых итераций: 18
Ошибка: 5.991884058270496e-08

CPU times: user 727 ms, sys: 49.4 ms, total: 777 ms
Wall time: 1.11 s
Метод градиентного спуска: 2
Ошибка: 6.250000385853205e-08

CPU times: user 578 ms, sys: 102 ms, total: 679 ms
Wall time: 566 ms
Двухшаговый метод градиентного спуска: 2
Ошибка: 6.250000385853205e-08

CPU times: user 578 ms, sys: 92.7 ms, total: 671 ms
Wall time: 581 ms
Метод бисопряженных градиентов: 2
Ошибка: 6.250000394149216e-08

